In [1]:
import matplotlib.pyplot as plt

class Graphe:
    def __init__(self):
        self.graphe = {}
        self.depart = None
        self.arrivee = None

    def ajouter_sommet(self, sommet):
        if sommet not in self.graphe:
            self.graphe[sommet] = set()

    def ajouter_arete(self, u, v):
        if u in self.graphe and v in self.graphe:
            self.graphe[u].add(v)
            self.graphe[v].add(u)
    
    def ajouter_arete(self, u, v):
        if u in self.graphe and v in self.graphe:
            self.graphe[u].add(v)
            self.graphe[v].add(u)

    def supprimer_arete(self, u, v):
        if u in self.graphe and v in self.graphe:
            self.graphe[u].discard(v)
            self.graphe[v].discard(u)

    def creer_graphe_depuis_fichier(self, chemin_fichier):
        with open(chemin_fichier, 'r') as fichier:
            lignes = fichier.readlines()
        
        n, m = map(int, lignes[0].split())
        matrice = [ligne.split() for ligne in lignes[1:]]
        
        for y in range(n):
            for x in range(m):
                if matrice[y][x] in {'1', '2', '3'}:
                    self.ajouter_sommet((x, y))
                    if matrice[y][x] == '2':
                        self.depart = (x, y)
                    if matrice[y][x] == '3':
                        self.arrivee = (x, y)
                
                # Ajouter les arêtes pour les sommets voisins dans le graphe
                if matrice[y][x] != '0':
                    if x > 0 and matrice[y][x-1] != '0':  # À gauche
                        self.ajouter_arete((x, y), (x - 1, y))
                    if y > 0 and matrice[y-1][x] != '0':  # En haut
                        self.ajouter_arete((x, y), (x, y - 1))
    
    def afficher_matrice(self):
        # Déterminer la taille de la matrice
        max_x = max(self.graphe, key=lambda x: x[0])[0] + 1
        max_y = max(self.graphe, key=lambda x: x[1])[1] + 1
        
        # Initialiser la matrice avec des zéros
        matrice = [['0' for _ in range(max_x)] for _ in range(max_y)]
        
        # Remplir la matrice avec les chemins (1), départ (2) et arrivée (3)
        for (x, y) in self.graphe:
            matrice[y][x] = '1'
        if self.depart:
            matrice[self.depart[1]][self.depart[0]] = '2'
        if self.arrivee:
            matrice[self.arrivee[1]][self.arrivee[0]] = '3'
        
        # Afficher la matrice
        for ligne in matrice:
            print(' '.join(ligne))

    def tracer_graphe(self):
        plt.figure(figsize=(8, 6))
        # Dessiner les arêtes
        for u in self.graphe:
            for v in self.graphe[u]:
                plt.plot([u[0], v[0]], [u[1], v[1]], 'b-', linewidth=1)  # lignes bleues pour les arêtes

        # Dessiner les sommets
        for u in self.graphe:
            plt.plot(u[0], u[1], 'ko', markersize=8)  # points noirs pour les sommets

        # Mettre en évidence les points de départ et d'arrivée
        if self.depart:
            plt.plot(self.depart[0], self.depart[1], 'go', markersize=10)  # Vert pour départ
        if self.arrivee:
            plt.plot(self.arrivee[0], self.arrivee[1], 'ro', markersize=10)  # Rouge pour arrivée

        plt.xlabel('X coordinate')
        plt.ylabel('Y coordinate')
        plt.title('Visualisation du Graphe')
        plt.gca().invert_yaxis()  # Inverser l'axe Y pour correspondre au format de matrice
        plt.grid(True)
        plt.show()
    


# Exemple d'utilisation

# Usage example
graphe = Graphe()
graphe.creer_graphe_depuis_fichier('exo1.txt')  # Assurez-vous que le chemin d'accès est correct
graphe.afficher_matrice()




1 2 1 1 1 1 1 1 0
1 0 1 1 1 1 0 1 1
1 1 1 0 0 1 0 1 1
1 0 1 1 1 1 3 1 1


In [ ]:
graphe.tracer_graphe()